# U2 - Utilities: Optimize Model
Author: George Gorospe, george.gorospe@nmaia.net (updated 1/15/2025)

## In this second utility notebook we use torch2TRT, a tool to convert PyTorch models to TensorRT models. This conversion increases the performance of the model during inference. This means that when live images collected from the camera are sent (60 per second) to the model, the model's processing time for each frame is accelerated.

In [ ]:
# Importing Required Libraries
# General use libraries
import os
from ipyfilechooser import FileChooser

# Custom library from Nvidia to accelerate inference
from torch2trt import torch2trt

# PyTorch libraries
import torch
import torchvision # Import the TorchVision library from PyTorch
from torchvision.models import ResNet18_Weights 
from torchvision.models import ResNet34_Weights 
from torchvision.models import ResNet50_Weights

# Static variables
optimized_model_folder = "/home/racer_core/Models/trt/"
# Model Output
output_dim = 2

## Next, select the model you would like to optimize.

In [ ]:
# Create and display a FileChooser widget
fc = FileChooser('/home/racer_core/Models')

# Set a file filter pattern (uses https://docs.python.org/3/library/fnmatch.html)
fc.filter_pattern = '*.pth'

display(fc)
# Change the title (use '' to hide)
fc.title = '<b>Choose a model for optimization</b>'

# Sample callback function
def change_title(chooser):
    chooser.title = '<b>Model Selected.</b>'

# Register callback function
fc.register_callback(change_title)

In [ ]:
# Inspecting Model:
model_name = fc.selected_filename
model_name = model_name.split(".")[-2] # extract the model name without file extention
model_file_path = fc.selected # extract full path to the model

# Building the file path and new filename for the optimized model
optimized_model_file_path = optimized_model_folder + model_name + "_TRT.pth"

# Model Name check: did we already optimize this model? if so, throw an error!
if os.path.isfile(optimized_model_file_path):
    raise Exception('Sorry, an optimized model with same name already exists, perhaps you already optimized this model?')

In [ ]:
# First Optimization Cell
# Warm starting the new model to be optimized - loading weights from trained model into untrained model
# Start by selecting your model structure, it should match your trained model, the default is listed
model_name = "Resnet 18"
model = torchvision.models.resnet18(weights=ResNet18_Weights.DEFAULT)
model.fc = torch.nn.Linear(512, output_dim)

# OPTIONAL: Uncomment one of these other models if you have used their structure in your training.
# Resnet 34
#model_name = "Resnet 34"
#model = torchvision.models.resnet34(pretrained=True)
#model.fc = torch.nn.Linear(512, output_dim)

# Resnet 50
#model_name = "Resnet 50"
#model = torch.hub.load("pytorch/vision", "resnet50", weights="IMAGENET1K_V2")
#model.fc = torch.nn.Linear(2048, output_dim)


model = model.cuda().eval().half()
model.load_state_dict(torch.load(model_file_path, weights_only=True))

In [ ]:
# Second Optimization Cell

# Example structure of the input data
data = torch.zeros((1, 3, 224, 224)).cuda().half()

# Model optimization via quantitization, or the reduction of overall model size by reducing the representation of model weights.
model_trt = torch2trt(model, [data], fp16_mode=True)

# Saving the new optimized model to disk with a new name
torch.save(model_trt.state_dict(), optimized_model_file_path)

# Print output
print("Successfully optimized Model!")
print(f"Optimized Model: {optimized_model_file_path}") 